<a href="https://colab.research.google.com/github/lamyse1/deep-learning-/blob/main/Week6/DL_Week6_Ex3_Notebook6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AAI612: Deep Learning & its Applications

*Notebook 6.4: How do you feel about that movie?*


In [1]:
import pandas as pd
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

df = pd.DataFrame()
df = pd.read_csv('https://raw.githubusercontent.com/harmanani/AAI612/main/Week6/data/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [2]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [3]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [4]:
# summarize size
print("Training data: ")
print(X.shape)
print(y.shape)

Training data: 
(50000,)
(50000,)


In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



tokenizer_obj = Tokenizer()
total_reviews = X_train + X_test
tokenizer_obj.fit_on_texts(total_reviews)

# pad sequences
max_length = 100  # try other options like mean

# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

In [6]:
print(vocab_size)

125602


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding

EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Build model...
Summary of the built model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Epoch 1/25
196/196 - 89s - 452ms/step - accuracy: 0.6872 - loss: 0.5711 - val_accuracy: 0.7756 - val_loss: 0.4680
Epoch 2/25
196/196 - 75s - 383ms/step - accuracy: 0.8507 - loss: 0.3664 - val_accuracy: 0.8317 - val_loss: 0.3868
Epoch 3/25
196/196 - 83s - 424ms/step - accuracy: 0.9024 - loss: 0.2552 - val_accuracy: 0.8295 - val_loss: 0.4047
Epoch 4/25
196/196 - 82s - 419ms/step - accuracy: 0.9387 - loss: 0.1757 - val_accuracy: 0.8262 - val_loss: 0.4365
Epoch 5/25
196/196 - 72s - 367ms/step - accuracy: 0.9621 - loss: 0.1153 - val_accuracy: 0.8232 - val_loss: 0.4868
Epoch 6/25
196/196 - 81s - 415ms/step - accuracy: 0.9752 - loss: 0.0748 - val_accuracy: 0.8176 - val_loss: 0.5685
Epoch 7/25
196/196 - 91s - 465ms/step - accuracy: 0.9866 - loss: 0.0452 - val_accuracy: 0.8147 - val_loss: 0.6658
Epoch 8/25
196/196 - 82s - 417ms/step - accuracy: 0.9906 - loss: 0.0311 - val_accuracy: 0.8102 - val_loss: 0.7440
Epoch 9/25
196/196 - 82s - 418ms/step - accuracy: 0.9942 - loss: 0.0202 - val_a

In [10]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

Testing...
196/196 ━━━━━━━━━━━━━━━━━━━━ 12s 61ms/step - accuracy: 0.7592 - loss: 1.6109
Test score: 1.4211193323135376
Test accuracy: 0.7870000004768372
Accuracy: 78.70%


In [11]:
#Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

#predict
model.predict(x=test_samples_tokens_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step


array([[9.9993253e-01],
       [9.1222060e-01],
       [9.0508407e-01],
       [4.1813055e-05],
       [9.7193921e-01],
       [1.2880486e-01],
       [7.9356986e-01],
       [1.2054891e-06]], dtype=float32)

In [12]:
#let us check how the model predicts
classes = model.predict(X_test_pad[:10], batch_size=128)
for i in range (0,10):
    if(classes[i] > 0.5 and y_test[i] == 1 or (classes[i] <= 0.5 and y_test[i] == 0)):
        print( classes[i], y_test[i], " Right prdiction")
    else :
        print( classes[i], y_test[i], " Wrong prdiction")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
[0.04335602] 1  Wrong prdiction
[4.7180663e-05] 1  Wrong prdiction
[0.9999869] 1  Right prdiction
[0.8344175] 1  Right prdiction
[0.9999933] 1  Right prdiction
[0.9982723] 1  Right prdiction
[0.9999722] 1  Right prdiction
[2.2676633e-05] 1  Wrong prdiction
[0.02094198] 1  Wrong prdiction
[0.999995] 1  Right prdiction


In [8]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('Build model...')

model = Sequential()
model.add(Embedding(top_words, 100, input_length=max_words))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Build model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
print('Train...')

model.fit(X_train, y_train, batch_size=128, epochs=12, validation_data=(X_test, y_test), verbose=2)

Train...
Epoch 1/12
196/196 - 347s - 2s/step - accuracy: 0.7973 - loss: 0.4545 - val_accuracy: 0.8053 - val_loss: 0.4376
Epoch 2/12
196/196 - 413s - 2s/step - accuracy: 0.8434 - loss: 0.3728 - val_accuracy: 0.8311 - val_loss: 0.4016
Epoch 3/12
196/196 - 375s - 2s/step - accuracy: 0.8533 - loss: 0.3522 - val_accuracy: 0.8452 - val_loss: 0.3675
Epoch 4/12
196/196 - 387s - 2s/step - accuracy: 0.8752 - loss: 0.3068 - val_accuracy: 0.8406 - val_loss: 0.3897
Epoch 5/12
196/196 - 382s - 2s/step - accuracy: 0.8773 - loss: 0.3017 - val_accuracy: 0.8437 - val_loss: 0.3929
Epoch 6/12
196/196 - 346s - 2s/step - accuracy: 0.8847 - loss: 0.2873 - val_accuracy: 0.8578 - val_loss: 0.3791
Epoch 7/12
196/196 - 410s - 2s/step - accuracy: 0.8876 - loss: 0.2768 - val_accuracy: 0.8553 - val_loss: 0.3658
Epoch 8/12
196/196 - 385s - 2s/step - accuracy: 0.8928 - loss: 0.2610 - val_accuracy: 0.8592 - val_loss: 0.3562
Epoch 9/12
196/196 - 375s - 2s/step - accuracy: 0.8978 - loss: 0.2589 - val_accuracy: 0.8560 - 

In [11]:
score, acc = model.evaluate(X_test, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)
print("Accuracy: %.2f%%" % (acc*100))

196/196 ━━━━━━━━━━━━━━━━━━━━ 53s 273ms/step - accuracy: 0.8576 - loss: 0.3811
Test score: 0.37341412901878357
Test accuracy: 0.8593199849128723
Accuracy: 85.93%


The time to train a GRU is less than LSTM network.